# Draft:  Binary fitting using AMICAL

***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data
- *import amical* University of Sydney interferometric package


Clone AMICAL repository on your computer:
git clone https://github.com/SydneyAstrophotonicInstrumentationLab/AMICAL.git  

Then, in the astropy conda environment, install AMICAL:
cd AMICAL/
pip install .


In [ ]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from astroquery.mast import Mast
from astroquery.mast import Observations
import multiprocessing
import amical

## Introduction
This notebook takes a calibrated OIFITS file and estimates the parameters of a best-fit binary model  It is closely based on Anthony Soulain's example script analyzing NIRISS AMI commissioning on the know binary AB Dor AC. 

In this use, AMICAL interfaces with the [CANDID (Companion Analysis and Non-Detection in Interferometric Data) analysis package](https://github.com/amerand/CANDID), which is included with AMICAL.  The  CANDID code is decribed by [Galenne et al. 2015](https://ui.adsabs.harvard.edu/link_gateway/2015A&A...579A..68G/doi:10.1051/0004-6361/201525917).    It utilizes model-fitting and chi-squared minimization.  See the [CANDID github page](https://github.com/amerand/CANDID) for further details.

### Defining terms

[OIFITS](https://doi.org/10.1051/0004-6361/201526405) files are the Optical Interferometry standard FITS files used by the community. A science target's observables are **calibrated** by the observables of a PSF calibrator star's image, to remove telescope/instrument effects (as far as possible).

**AMICAL** is **A**perture **M**asking **I**nterferometry **C**alibration and **A**nalysis **L**ibrary.  It is developed by the Sydney Astrophotonic Instrumentation Lab [![version](http://img.shields.io/badge/AMICAL-v0.5dev-orange.svg?style=flat)](https://github.com/SydneyAstrophotonicInstrumentationLab/AMICAL.git)



***

## Loading data
Load the input, a science target's calibrated oifits file, from **longURL**.  This file needs to have interferometric observables corresponding to all possible baselines in the non-redundant mask, and their "straight" standard deviations (without consideration for linear dependence within a set of variables).

**Developer note** Use our tools to look at some of the fits tables: closure phase and fringe amplitude in particlar.  More visuals during the reduction later on here...

The example dataset comes with a NIRISS simulated dataset representing a faint binary star 
- Magnitude difference: dm = 6 mag at 4.3 μm (a flux ratio of 251.2)
- Sub-λ/D separation sep = 147.7 mas
- Position angle theta = 46.6 degrees 
- Filter F430M


In [ ]:
sep = 147.7  # binary separation [mas]
theta = 46.6  # position angle (pa) [deg]
dm = 6.0  # contrast ratio [mag]

In [ ]:
# Download the products
# Observations.download_products(yourProd, mrp_only=False, cache=False)

## File Information
Load the input, a science target's calibrated oifits file, from **longURL**.  This file needs to have interferometric observables corresponding to all possible baselines in the non-redundant mask, and their "straight" standard deviations (without consideration for linear dependence within a set of variables).
OIFITS files are multi-extension FITS binary tables.  The sample input file contains HDU's

In [ ]:
%%html
<style>
table {float:left}
</style>


|No.| Name     |  Ver  | Type          | Cards|  Dimensions    |
|:---:|:---------------|:---:|:-------------|:-----:|:---------|
  0 | PRIMARY       | 1 | PrimaryHDU   |  18 | None
  1 | OI_WAVELENGTH | 1 | BinTableHDU  |  17 | 1R x 2C
  2 | OI_TARGET     | 1 | BinTableHDU  |  56 | 1R x 17C 
  3 | OI_ARRAY      | 1 | BinTableHDU  |  32 | 7R x 7C 
  4 | OI_VIS2       | 1 | BinTableHDU  |  38 | 21R x 10C
  5 | OI_T3         | 1 | BinTableHDU  |  50 | 35R x 14C
  

**Developer note** Final HDU's will be similar, but is 2_niriss_ami_binary.py output.  For initial development a canned other oifits with slightly different structure is used.  The essential HDUs exist in both input files, to enable notebook development.

## Sections (xN)

The meat of your notebook should be sub-divided into sections with useful names to whatever extent makes sense base on the content. Break sections up with standard Markdown syntax headings:

```
# Section 1



### Sub-section 1a

More detailed info about Section 1

## Section 2

A complete thought that's as important as section 1 but doesn't need sub-sections.
```

In [ ]:
# Your input data is an oifits file (or a list of oifits files).
inputdata = 'example_fakebinary_NIRISS.oifits'

In [ ]:
# Set up parameters for the binary serch grid
param_candid = {'rmin': 20,  # inner radius of the grid
                'rmax': 250,  # outer radius of the grid
                'step': 50,  # grid sampling
                'ncore': multiprocessing.cpu_count()  # core for multiprocessing
                }

In [ ]:
# Utilize the provided interface to the CANDID package
""" This function is an user friendly interface between the users of amical
    pipeline and the CANDID analysis package (https://github.com/amerand/CANDID).

    Parameters:
    -----------
    `input_data` {str or list}:
        oifits file names or list of oifits files,\n
    `step` {int}:
        step used to compute the binary grid positions,\n
    `rmin`, `rmax` {float}:
        Bounds of the grid [mas],\n
    `diam` {float}:
        Stellar diameter of the primary star [mas] (default=0),\n
    `obs` {list}:
        List of observables to be fitted (default: ['cp', 'v2']),\n
    `doNotFit` {list}:
        Parameters not fitted (default: ['diam*']),\n
    `verbose` {boolean}:
        print some informations {default: False}.

    Outputs:
    --------
    `res` {dict}:
        Dictionary of the results ('best'), uncertainties ('uncer'),
        reduced chi2 ('chi2') and sigma detection ('nsigma').
"""

fit1 = amical.candid_grid(inputdata, **param_candid, diam=20, doNotFit=[])

In [ ]:
#  Find detection limits using the fake injection method
cr_candid = amical.candid_cr_limit(
            inputdata, **param_candid, fitComp=fit1['comp'])

## Exercises
Most notebooks are trying to convey _some_ information to their reader. It's often a good idea, if possible, to include in the notebook something where there is not an included answer so that the reader can cement in their mind how whatever it is they were supposed to learn from that notebook. If you do have exercise(s), be sure to leave a blank code cell underneath to show the user that it's meant for them to try it out right there. You may also want to include a "solutions" notebook next to your main notebook for the user to look at after they have finished their attempt.

## Aditional Resources
While this isn't always necessary, sometimes you want to provide some more resources for the reader who wants to learn something beyond what's in the notebook. Sometimes these don't exist, but if they do, it's good to put them at the end to give the reader somewhere else to go. Usually a list of links using markdown bullet-plus-link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/exo/ExoMast/html/exomast.html)

## About this notebook
**Author:** Anand Sivaramakrishnan's transcription of an analysis script written by Anthony Soulain (University of Sydney) that runs the CANDID interferometric package  
**Updated On:** 2020-09-17

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 